[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ma7555/kerasgen/blob/master/examples/triplet_loss.ipynb)

In [12]:
!git clone https://github.com/ma7555/kerasgen
%cd kerasgen
%pip install .
%pip install tensorflow-addons

c:\Users\ma7mo\Documents\GitHub\kerasgen\examples\kerasgen


fatal: destination path 'kerasgen' already exists and is not an empty directory.
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


Processing c:\users\ma7mo\documents\github\kerasgen\examples\kerasgen
  Created wheel for kerasgen: filename=kerasgen-0.1.2-py3-none-any.whl size=8754 sha256=e42ab5abad6c585436908e7f1c945478ed6630ec5956177f3f4852396c886951
  Stored in directory: C:\Users\ma7mo\AppData\Local\Temp\pip-ephem-wheel-cache-e761ru5_\wheels\cb\c6\cd\2ea5f4585a7b6a76b39fed4d4bd1566142fab984d4d2071d92
Successfully built kerasgen


In [9]:
from kerasgen.balanced_image_dataset import balanced_image_dataset_from_directory
import tensorflow as tf
import tensorflow_addons as tfa
import os

c:\ProgramData\Anaconda3\envs\kerasgen\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [10]:
directory = '../sample' if os.path.exists('../sample') else './sample'

train_ds = balanced_image_dataset_from_directory(
    directory, num_classes_per_batch=2,
    num_images_per_class=4, image_size=(256, 256),
    validation_split=0.2, subset='training', seed=555,
    safe_triplet=True)

val_ds = balanced_image_dataset_from_directory(
    directory, num_classes_per_batch=2,
    num_images_per_class=4, image_size=(256, 256),
    validation_split=0.2, subset='validation', seed=555,
    safe_triplet=True)

Found 51 files belonging to 5 classes.
Using 41 files for training.
Found 51 files belonging to 5 classes.
Using 10 files for validation.


In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings

])

In [12]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())

In [13]:
# Train the network
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5)

Epoch 1/5
6/6 [==============================] - 10s 1s/step - loss: 0.9559 - val_loss: 0.5348
Epoch 2/5
6/6 [==============================] - 5s 790ms/step - loss: 0.7503 - val_loss: 0.4937
Epoch 3/5
6/6 [==============================] - 4s 688ms/step - loss: 0.4935 - val_loss: 0.5610
Epoch 4/5
6/6 [==============================] - 5s 811ms/step - loss: 0.3719 - val_loss: 0.5519
Epoch 5/5
6/6 [==============================] - 4s 728ms/step - loss: 0.2309 - val_loss: 0.5484
